In [1]:
from google.colab import drive
drive.mount('/content/drive')

rawdata_path = '/content/drive/MyDrive/my_project/my_project1/rawdata'
data_path = '/content/drive/MyDrive/my_project/my_project1/data'
final_result = '/content/drive/MyDrive/my_project/my_project1/result/reg(인구)'
final_img = '/content/drive/MyDrive/my_project/my_project1/result/img'
# rawdata_path = 'rawdata/'
# data_path = 'data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# # 1. 나눔 폰트 다운
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
from mpl_toolkits.axes_grid1 import make_axes_locatable

import geopandas as gpd

import statsmodels.api as sm
import re
from io import StringIO

from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

In [4]:
year_list = ['2022', '2023', '2024']

for year in year_list:
    globals()[f'temp_{year}'] = pd.read_csv(f'{data_path}/final/final_{year}.csv')

    globals()[f'temp_{year}'][f'solo_20s_{year}'] = globals()[f'temp_{year}'].apply(lambda x: (x[f'solo_20_{year}'] + x[f'solo_25_{year}']) / 2, axis = 1)
    globals()[f'temp_{year}'][f'solo_30s_{year}'] = globals()[f'temp_{year}'].apply(lambda x: (x[f'solo_30_{year}'] + x[f'solo_35_{year}']) / 2, axis = 1)

    globals()[f'temp_{year}'][f'solo_all_ratio'] = globals()[f'temp_{year}'].apply(lambda x: x[f'solo_all_{year}'] / x[f'total_all_{year}'] * 100, axis=1)
    globals()[f'temp_{year}'][f'solo_20s_ratio'] = globals()[f'temp_{year}'].apply(lambda x: x[f'solo_20s_{year}'] / x[f'solo_all_{year}'] * 100, axis=1)
    globals()[f'temp_{year}'][f'solo_30s_ratio'] = globals()[f'temp_{year}'].apply(lambda x: x[f'solo_30s_{year}'] / x[f'solo_all_{year}'] * 100, axis=1)
    globals()[f'temp_{year}'] = globals()[f'temp_{year}'].replace('-', '0')



In [5]:
depend_list = ['solo_all_ratio', 'solo_20s_ratio', 'solo_30s_ratio']
depend_name = ['all', '20', '30']

In [6]:
# reg_col = ['ADM_CD', 'solo_all_ratio', 'solo_20s_ratio',
#            '네일숍', '미용실', '애완동물',
#            '분식전문점', '한식음식점', '양식음식점', '일식음식점', '중식음식점', '치킨전문점',
#            '커피-음료', '패스트푸드점', '제과점', '호프-간이주점',
#            '유통업체수', '대형마트', '쇼핑업체수', '편의점', '슈퍼마켓',
#            '지가변동률', '지가',
#            '사업체수', '종사자수', '금융기관',
#            '대학교', '기타대학', '면적', '월평균소득',
#            '총상주인구', '20대상주인구', '총직장인구', '20대직장인구']

In [7]:
var_col = ['네일숍', '미용실', '애완동물',
           '분식전문점', '한식음식점', '양식음식점', '일식음식점', '중식음식점', '치킨전문점',
           '커피-음료', '패스트푸드점', '제과점', '호프-간이주점',
            '대형마트', '쇼핑업체수', '편의점', '슈퍼마켓'
           ]

nocal_col = ['사업체수', '종사자수', '지가변동률', '지가', '대학교', '기타대학' ]

drop_col = [ '면적', '월평균소득', '총상주인구', '20대상주인구', '총직장인구', '20대직장인구', '유통업체수',]

reg_col = ['ADM_CD'] + depend_list + var_col + nocal_col

In [8]:
df_2022 = temp_2022.copy()

In [9]:
columns_2022 = [col.replace('_2022', '') for col in df_2022.columns]
columns_2023 = [col.replace('_2023', '') for col in columns_2022]
columns_2024 = [col.replace('_2024', '') for col in columns_2023]
df_2022.columns = columns_2024
df_2022.columns

Index(['ADM_CD', 'total_all', 'solo_all', 'total_20', 'solo_20', 'total_25',
       'solo_25', 'total_30', 'solo_30', 'total_35',
       ...
       '총매출금액_커피-음료', '총매출금액_패스트푸드점', '총매출금액_편의점', '총매출금액_한식음식점',
       '총매출금액_호프-간이주점', 'solo_20s', 'solo_30s', 'solo_all_ratio',
       'solo_20s_ratio', 'solo_30s_ratio'],
      dtype='object', length=133)

In [10]:
reg_2022 = df_2022[reg_col + ['total_all']]
for var in var_col:
        reg_2022[var] = reg_2022[var] / reg_2022['total_all'] * 100

reg_2022.drop(columns = 'total_all', inplace=True)
reg_2022

,ADM_CD,solo_all_ratio,solo_20s_ratio,solo_30s_ratio,네일숍,미용실,애완동물,분식전문점,한식음식점,양식음식점,...,쇼핑업체수,편의점,슈퍼마켓,사업체수,종사자수,지가변동률,지가,대학교,기타대학,total_all
0,11010530,23.623853,10.868393,12.108954,13.0,30.0,10.0,127.0,413.0,183.0,...,0,11.0,0.0,4994,56046,4.858500,96.517917,0.0,0.0,7848.0
1,11010540,34.857383,10.108303,11.371841,1.0,0.0,0.0,17.0,73.0,85.0,...,0,2.0,0.0,970,5118,4.673571,96.461429,0.0,0.0,2384.0
2,11010550,16.783677,11.601307,10.947712,0.0,10.0,6.0,14.0,41.0,37.0,...,0,3.0,0.0,908,4140,5.295000,96.764000,1.0,0.0,9116.0
3,11010560,10.436593,8.073566,8.135910,2.0,16.0,3.0,15.0,56.0,19.0,...,0,3.0,0.0,1286,4294,5.151000,96.442000,0.0,0.0,15369.0
4,11030740,28.773407,11.130908,15.041260,11.0,46.0,14.0,27.0,197.0,210.0,...,0,11.0,3.0,3213,20440,6.127000,95.366000,0.0,0.0,18531.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,11010600,28.133635,8.928571,12.142857,0.0,8.0,1.0,21.0,67.0,44.0,...,0,3.0,0.0,977,15449,4.457000,96.575000,0.0,0.0,3981.0
418,11010610,49.126157,12.002990,8.505232,11.0,35.0,11.0,240.0,1173.0,316.0,...,7,31.0,1.0,17095,106826,4.757000,96.444172,1.0,0.0,6809.0
419,11220630,13.926386,11.276252,12.665590,53.0,227.0,59.0,103.0,525.0,105.0,...,1,65.0,1.0,2720,16795,6.174000,95.269000,0.0,0.0,22224.0
420,11090700,24.407020,12.764194,9.997928,52.0,309.0,61.0,112.0,654.0,30.0,...,0,51.0,0.0,2297,7190,4.930000,97.841000,0.0,1.0,19773.0


,ADM_CD,solo_all_ratio,solo_20s_ratio,solo_30s_ratio,네일숍,미용실,애완동물,분식전문점,한식음식점,양식음식점,...,대형마트,쇼핑업체수,편의점,슈퍼마켓,사업체수,종사자수,지가변동률,지가,대학교,기타대학
0,11010530,23.623853,10.868393,12.108954,1.656473,3.822630,1.274210,16.182467,52.624873,23.318043,...,0.0,0.000000,1.401631,0.000000,636.340469,7141.437309,4.858500,96.517917,0.0,0.0
1,11010540,34.857383,10.108303,11.371841,0.419463,0.000000,0.000000,7.130872,30.620805,35.654362,...,0.0,0.000000,0.838926,0.000000,406.879195,2146.812081,4.673571,96.461429,0.0,0.0
2,11010550,16.783677,11.601307,10.947712,0.000000,1.096972,0.658183,1.535761,4.497587,4.058798,...,0.0,0.000000,0.329092,0.000000,99.605090,454.146556,5.295000,96.764000,1.0,0.0
3,11010560,10.436593,8.073566,8.135910,0.130132,1.041057,0.195198,0.975991,3.643698,1.236255,...,0.0,0.000000,0.195198,0.000000,83.674930,279.393584,5.151000,96.442000,0.0,0.0
4,11030740,28.773407,11.130908,15.041260,0.593600,2.482327,0.755491,1.457018,10.630835,11.332362,...,0.0,0.000000,0.593600,0.161891,173.385138,1103.016567,6.127000,95.366000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,11010600,28.133635,8.928571,12.142857,0.000000,2.009545,0.251193,5.275057,16.829942,11.052499,...,0.0,0.000000,0.753580,0.000000,245.415725,3880.683245,4.457000,96.575000,0.0,0.0
418,11010610,49.126157,12.002990,8.505232,1.615509,5.140256,1.615509,35.247467,172.271993,46.409164,...,0.0,1.028051,4.552798,0.146864,2510.647672,15688.941107,4.757000,96.444172,1.0,0.0
419,11220630,13.926386,11.276252,12.665590,2.384809,10.214183,2.654788,4.634629,23.623110,4.724622,...,0.0,0.044996,2.924766,0.044996,122.390209,755.714543,6.174000,95.269000,0.0,0.0
420,11090700,24.407020,12.764194,9.997928,2.629849,15.627371,3.085015,5.664290,33.075406,1.517220,...,0.0,0.000000,2.579275,0.000000,116.168513,363.627168,4.930000,97.841000,0.0,1.0


In [12]:
df_2023 = temp_2023.copy()
df_2023.head()

,ADM_CD,total_all_2023,solo_all_2023,total_20_2023,solo_20_2023,total_25_2023,solo_25_2023,total_30_2023,solo_30_2023,total_35_2023,...,총매출금액_커피-음료_2023,총매출금액_패스트푸드점_2023,총매출금액_편의점_2023,총매출금액_한식음식점_2023,총매출금액_호프-간이주점_2023,solo_20s_2023,solo_30s_2023,solo_all_ratio,solo_20s_ratio,solo_30s_ratio
0,11010530,7845.0,1869.0,583.0,172.0,686.0,241.0,778.0,222.0,757.0,...,1.187346e+10,902523398.0,5.084405e+09,3.660644e+10,3.245160e+09,206.5,226.0,23.824092,11.048689,12.092028
1,11010540,2370.0,800.0,143.0,68.0,172.0,85.0,193.0,92.0,211.0,...,6.900190e+09,111870248.0,5.324073e+08,7.375948e+09,2.476535e+08,76.5,91.5,33.755274,9.562500,11.437500
2,11010550,9116.0,1549.0,1147.0,213.0,841.0,154.0,711.0,187.0,652.0,...,4.776125e+08,189999970.0,8.978345e+08,7.587584e+08,2.295424e+07,183.5,169.5,16.992102,11.846352,10.942544
3,11010560,15369.0,1617.0,1269.0,136.0,1173.0,123.0,1172.0,143.0,1075.0,...,9.633955e+08,284394280.0,3.962739e+08,2.456528e+09,2.872706e+08,129.5,132.0,10.521179,8.008658,8.163265
4,11030740,18463.0,5314.0,1138.0,442.0,1823.0,743.0,2384.0,881.0,2035.0,...,4.049663e+09,415945610.0,4.542987e+09,2.130260e+10,3.918752e+09,592.5,792.0,28.781888,11.149793,14.904027


In [13]:
columns_2022 = [col.replace('_2022', '') for col in df_2023.columns]
columns_2023 = [col.replace('_2023', '') for col in columns_2022]
columns_2024 = [col.replace('_2024', '') for col in columns_2023]
df_2023.columns = columns_2024
df_2023.columns

Index(['ADM_CD', 'total_all', 'solo_all', 'total_20', 'solo_20', 'total_25',
       'solo_25', 'total_30', 'solo_30', 'total_35',
       ...
       '총매출금액_커피-음료', '총매출금액_패스트푸드점', '총매출금액_편의점', '총매출금액_한식음식점',
       '총매출금액_호프-간이주점', 'solo_20s', 'solo_30s', 'solo_all_ratio',
       'solo_20s_ratio', 'solo_30s_ratio'],
      dtype='object', length=133)

In [14]:
reg_2023 = df_2023[reg_col + ['total_all']]
reg_2023

,ADM_CD,solo_all_ratio,solo_20s_ratio,solo_30s_ratio,네일숍,미용실,애완동물,분식전문점,한식음식점,양식음식점,...,쇼핑업체수,편의점,슈퍼마켓,사업체수,종사자수,지가변동률,지가,대학교,기타대학,total_all
0,11010530,23.824092,11.048689,12.092028,13.0,32.0,8.0,124.0,414.0,182.0,...,0,11.0,0.0,4994,56046,7.902167,98.973167,0.0,0.0,7845.0
1,11010540,33.755274,9.562500,11.437500,2.0,0.0,0.0,17.0,73.0,85.0,...,0,1.0,0.0,970,5118,7.840571,99.057857,0.0,0.0,2370.0
2,11010550,16.992102,11.846352,10.942544,0.0,10.0,7.0,14.0,38.0,37.0,...,0,4.0,0.0,908,4140,8.004000,98.822000,1.0,0.0,9116.0
3,11010560,10.521179,8.008658,8.163265,2.0,16.0,3.0,14.0,55.0,18.0,...,0,3.0,0.0,1286,4294,8.009000,98.655000,0.0,0.0,15369.0
4,11030740,28.781888,11.149793,14.904027,12.0,46.0,12.0,26.0,202.0,217.0,...,0,11.0,3.0,3213,20440,10.155000,98.676000,0.0,0.0,18463.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,11010600,28.862095,9.442994,12.358573,0.0,8.0,1.0,20.0,67.0,44.0,...,0,4.0,0.0,977,15449,7.389000,98.986000,0.0,0.0,3981.0
418,11010610,51.245969,12.514302,8.195080,10.0,34.0,13.0,238.0,1193.0,316.0,...,7,34.0,1.0,17095,106826,7.914103,99.068069,1.0,0.0,6822.0
419,11220630,14.020878,11.360719,12.885109,55.0,234.0,63.0,93.0,512.0,118.0,...,1,69.0,1.0,2720,16795,10.086000,98.672000,0.0,0.0,22224.0
420,11090700,23.520569,13.645079,10.161909,55.0,308.0,69.0,107.0,631.0,26.0,...,0,51.0,0.0,2297,7190,7.019000,99.601000,0.0,1.0,19957.0


In [15]:
for var in var_col:
        reg_2023[var] = reg_2023[var] / reg_2023['total_all'] * 100

reg_2023.drop(columns = 'total_all', inplace=True)
reg_2023

,ADM_CD,solo_all_ratio,solo_20s_ratio,solo_30s_ratio,네일숍,미용실,애완동물,분식전문점,한식음식점,양식음식점,...,대형마트,쇼핑업체수,편의점,슈퍼마켓,사업체수,종사자수,지가변동률,지가,대학교,기타대학
0,11010530,23.824092,11.048689,12.092028,1.657106,4.079031,1.019758,15.806246,52.772467,23.199490,...,0.0,0.000000,1.402167,0.000000,636.583811,7144.168260,7.902167,98.973167,0.0,0.0
1,11010540,33.755274,9.562500,11.437500,0.843882,0.000000,0.000000,7.172996,30.801688,35.864979,...,0.0,0.000000,0.421941,0.000000,409.282700,2159.493671,7.840571,99.057857,0.0,0.0
2,11010550,16.992102,11.846352,10.942544,0.000000,1.096972,0.767881,1.535761,4.168495,4.058798,...,0.0,0.000000,0.438789,0.000000,99.605090,454.146556,8.004000,98.822000,1.0,0.0
3,11010560,10.521179,8.008658,8.163265,0.130132,1.041057,0.195198,0.910925,3.578632,1.171189,...,0.0,0.000000,0.195198,0.000000,83.674930,279.393584,8.009000,98.655000,0.0,0.0
4,11030740,28.781888,11.149793,14.904027,0.649949,2.491469,0.649949,1.408222,10.940801,11.753236,...,0.0,0.000000,0.595786,0.162487,174.023723,1107.079023,10.155000,98.676000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,11010600,28.862095,9.442994,12.358573,0.000000,2.009545,0.251193,5.023863,16.829942,11.052499,...,0.0,0.000000,1.004773,0.000000,245.415725,3880.683245,7.389000,98.986000,0.0,0.0
418,11010610,51.245969,12.514302,8.195080,1.465846,4.983876,1.905600,34.887130,174.875403,46.320727,...,0.0,1.026092,4.983876,0.146585,2505.863383,15659.044269,7.914103,99.068069,1.0,0.0
419,11220630,14.020878,11.360719,12.885109,2.474802,10.529158,2.834773,4.184665,23.038157,5.309575,...,0.0,0.044996,3.104752,0.044996,122.390209,755.714543,10.086000,98.672000,0.0,0.0
420,11090700,23.520569,13.645079,10.161909,2.755925,15.433181,3.457433,5.361527,31.617979,1.302801,...,0.0,0.000000,2.555494,0.000000,115.097460,360.274590,7.019000,99.601000,0.0,1.0


In [16]:
df_2024 = temp_2024.copy()
df_2024.head()

,ADM_CD,total_all_2024,solo_all_2024,total_20_2024,solo_20_2024,total_25_2024,solo_25_2024,total_30_2024,solo_30_2024,total_35_2024,...,총매출금액_커피-음료_2023,총매출금액_패스트푸드점_2023,총매출금액_편의점_2023,총매출금액_한식음식점_2023,총매출금액_호프-간이주점_2023,solo_20s_2024,solo_30s_2024,solo_all_ratio,solo_20s_ratio,solo_30s_ratio
0,11010530,8934.0,2200.0,569.0,182.0,705.0,252.0,791.0,262.0,749.0,...,1.187346e+10,902523398.0,5.084405e+09,3.660644e+10,3.245160e+09,217.0,242.5,24.625028,9.863636,11.022727
1,11010540,2579.0,911.0,311.0,152.0,176.0,88.0,186.0,87.0,205.0,...,6.900190e+09,111870248.0,5.324073e+08,7.375948e+09,2.476535e+08,120.0,88.0,35.323769,13.172338,9.659715
2,11010550,9226.0,1582.0,1119.0,204.0,865.0,161.0,687.0,177.0,637.0,...,4.776125e+08,189999970.0,8.978345e+08,7.587584e+08,2.295424e+07,182.5,163.5,17.147193,11.536030,10.335019
3,11010560,15667.0,1646.0,1238.0,131.0,1207.0,128.0,1133.0,137.0,1051.0,...,9.633955e+08,284394280.0,3.962739e+08,2.456528e+09,2.872706e+08,129.5,125.0,10.506159,7.867558,7.594168
4,11030740,19241.0,5509.0,1111.0,439.0,1882.0,760.0,2315.0,871.0,1991.0,...,4.049663e+09,415945610.0,4.542987e+09,2.130260e+10,3.918752e+09,599.5,782.5,28.631568,10.882193,14.204030


In [17]:
columns_2022 = [col.replace('_2022', '') for col in df_2024.columns]
columns_2023 = [col.replace('_2023', '') for col in columns_2022]
columns_2024 = [col.replace('_2024', '') for col in columns_2023]
df_2024.columns = columns_2024
df_2024.columns

Index(['ADM_CD', 'total_all', 'solo_all', 'total_20', 'solo_20', 'total_25',
       'solo_25', 'total_30', 'solo_30', 'total_35',
       ...
       '총매출금액_커피-음료', '총매출금액_패스트푸드점', '총매출금액_편의점', '총매출금액_한식음식점',
       '총매출금액_호프-간이주점', 'solo_20s', 'solo_30s', 'solo_all_ratio',
       'solo_20s_ratio', 'solo_30s_ratio'],
      dtype='object', length=133)

In [18]:
reg_2024 = df_2024[reg_col  + ['total_all']]
reg_2024

,ADM_CD,solo_all_ratio,solo_20s_ratio,solo_30s_ratio,네일숍,미용실,애완동물,분식전문점,한식음식점,양식음식점,...,쇼핑업체수,편의점,슈퍼마켓,사업체수,종사자수,지가변동률,지가,대학교,기타대학,total_all
0,11010530,24.625028,9.863636,11.022727,14.0,35.0,10.0,122.0,407.0,187.0,...,0,12.0,0.0,4994,56046,8.764667,100.182750,0.0,0.0,8934.0
1,11010540,35.323769,13.172338,9.659715,1.0,0.0,1.0,17.0,76.0,85.0,...,0,1.0,0.0,970,5118,8.613000,100.133714,0.0,0.0,2579.0
2,11010550,17.147193,11.536030,10.335019,0.0,10.0,8.0,13.0,36.0,38.0,...,0,4.0,0.0,908,4140,9.032000,100.209000,1.0,0.0,9226.0
3,11010560,10.506159,7.867558,7.594168,3.0,15.0,4.0,13.0,53.0,20.0,...,0,4.0,0.0,1286,4294,9.204000,100.254000,0.0,0.0,15667.0
4,11030740,28.631568,10.882193,14.204030,14.0,48.0,13.0,24.0,199.0,236.0,...,0,11.0,3.0,3213,20440,11.399000,100.192000,0.0,0.0,19241.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,11010600,29.180084,8.870968,12.011885,0.0,8.0,1.0,19.0,67.0,43.0,...,0,4.0,0.0,977,15449,8.215000,100.184000,0.0,0.0,4037.0
418,11010610,48.905192,11.020078,9.381491,12.0,32.0,20.0,235.0,1219.0,320.0,...,7,29.0,1.0,17095,106826,8.712793,100.158517,1.0,0.0,8860.0
419,11220630,13.962063,10.738148,12.281260,61.0,230.0,61.0,85.0,524.0,118.0,...,1,70.0,1.0,2720,16795,11.430000,100.170000,0.0,0.0,22511.0
420,11090700,25.509848,12.617279,9.470289,62.0,304.0,73.0,93.0,601.0,23.0,...,0,58.0,0.0,2297,7190,7.376000,100.144000,0.0,1.0,20055.0


In [19]:
for var in var_col:
        reg_2024[var] = reg_2024[var] / reg_2024['total_all'] * 100

reg_2024.drop(columns = 'total_all', inplace=True)
reg_2024

,ADM_CD,solo_all_ratio,solo_20s_ratio,solo_30s_ratio,네일숍,미용실,애완동물,분식전문점,한식음식점,양식음식점,...,대형마트,쇼핑업체수,편의점,슈퍼마켓,사업체수,종사자수,지가변동률,지가,대학교,기타대학
0,11010530,24.625028,9.863636,11.022727,1.567047,3.917618,1.119319,13.655697,45.556302,20.931274,...,0.0,0.000000,1.343183,0.000000,558.988135,6273.337811,8.764667,100.182750,0.0,0.0
1,11010540,35.323769,13.172338,9.659715,0.387747,0.000000,0.387747,6.591702,29.468786,32.958511,...,0.0,0.000000,0.387747,0.000000,376.114773,1984.490112,8.613000,100.133714,0.0,0.0
2,11010550,17.147193,11.536030,10.335019,0.000000,1.083893,0.867115,1.409061,3.902016,4.118795,...,0.0,0.000000,0.433557,0.000000,98.417516,448.731845,9.032000,100.209000,1.0,0.0
3,11010560,10.506159,7.867558,7.594168,0.191485,0.957426,0.255314,0.829770,3.382907,1.276569,...,0.0,0.000000,0.255314,0.000000,82.083360,274.079275,9.204000,100.254000,0.0,0.0
4,11030740,28.631568,10.882193,14.204030,0.727613,2.494673,0.675641,1.247336,10.342498,12.265475,...,0.0,0.000000,0.571696,0.155917,166.987163,1062.314849,11.399000,100.192000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,11010600,29.180084,8.870968,12.011885,0.000000,1.981670,0.247709,4.706465,16.596483,10.651474,...,0.0,0.000000,0.990835,0.000000,242.011395,3826.851622,8.215000,100.184000,0.0,0.0
418,11010610,48.905192,11.020078,9.381491,1.354402,3.611738,2.257336,26.523702,137.584650,36.117381,...,0.0,0.790068,3.273138,0.112867,1929.458239,12057.110609,8.712793,100.158517,1.0,0.0
419,11220630,13.962063,10.738148,12.281260,2.709786,10.217227,2.709786,3.775932,23.277509,5.241882,...,0.0,0.044423,3.109591,0.044423,120.829817,746.079694,11.430000,100.170000,0.0,0.0
420,11090700,25.509848,12.617279,9.470289,3.091498,15.158315,3.639990,4.637248,29.967589,1.146846,...,0.0,0.000000,2.892047,0.000000,114.535029,358.514086,7.376000,100.144000,0.0,1.0


# 데이터 정규화

In [20]:
### 데이터 정규화

year_list = ['2022', '2023', '2024']

for year in year_list:
    globals()[f'reg_{year}'] = globals()[f'reg_{year}'].replace('-', '0')

    std_scaler = StandardScaler()
    df_std = std_scaler.fit_transform(globals()[f'reg_{year}'])
    df_std = pd.DataFrame(df_std)
    df_std.columns = globals()[f'reg_{year}'].columns
    df_std.drop(columns = 'ADM_CD', inplace = True)

    globals()[f'reg_{year}_temp'] = df_std.copy()

reg_2022 = reg_2022_temp.copy()
reg_2023 = reg_2023_temp.copy()
reg_2024 = reg_2024_temp.copy()


In [21]:
reg_2022

,solo_all_ratio,solo_20s_ratio,solo_30s_ratio,네일숍,미용실,애완동물,분식전문점,한식음식점,양식음식점,일식음식점,...,대형마트,쇼핑업체수,편의점,슈퍼마켓,사업체수,종사자수,지가변동률,지가,대학교,기타대학
0,0.415566,0.085282,0.198901,0.056452,-0.709275,-0.348807,2.276832,1.256923,2.508037,1.311383,...,-0.301346,-0.306233,-0.454019,-0.329860,1.053483,2.006377,-0.003407,-0.441216,-0.288565,-0.223050
1,1.502040,-0.042339,-0.003637,-0.887900,-1.308291,-1.355888,0.531416,0.342283,4.130139,0.129319,...,-0.301346,-0.306233,-0.717320,-0.329860,0.513153,0.371646,-0.222565,-0.487585,-0.288565,-0.223050
2,-0.245995,0.208340,-0.120176,-1.208124,-1.136392,-0.835688,-0.547488,-0.743577,-0.024359,-0.595684,...,-0.301346,-0.306233,-0.955882,-0.329860,-0.210407,-0.182360,0.513886,-0.239215,2.543395,-0.223050
3,-0.859866,-0.383977,-0.892781,-1.108779,-1.145155,-1.201612,-0.655428,-0.779071,-0.395495,-0.628621,...,-0.301346,-0.306233,-1.018533,-0.329860,-0.247919,-0.239556,0.343233,-0.503533,-0.288565,-0.223050
4,0.913616,0.129359,1.004618,-0.754961,-0.919304,-0.758780,-0.562672,-0.488637,0.932041,-0.049448,...,-0.301346,-0.306233,-0.832113,1.569775,-0.036672,0.030014,1.499884,-1.386783,-0.288565,-0.223050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,0.851739,-0.240419,0.208217,-1.208124,-0.993390,-1.157356,0.173559,-0.230960,0.895242,-0.212619,...,-0.301346,-0.306233,-0.757255,-0.329860,0.132944,0.939139,-0.479222,-0.394358,-0.288565,-0.223050
418,2.882073,0.275783,-0.791302,0.025179,-0.502800,-0.079059,5.953130,6.230277,5.544287,3.881441,...,-0.301346,4.023984,1.020475,1.393454,5.467051,4.803959,-0.123694,-0.501750,2.543395,-0.223050
419,-0.522344,0.153762,0.351849,0.612474,0.292296,0.742343,0.050066,0.051411,0.063190,0.178237,...,-0.301346,-0.116705,0.258686,0.198130,-0.156754,-0.083657,1.555584,-1.466407,-0.288565,-0.223050
420,0.491312,0.403592,-0.381150,0.799541,1.140555,1.082376,0.248615,0.444313,-0.358551,0.011011,...,-0.301346,-0.306233,0.097024,-0.329860,-0.171404,-0.211987,0.081327,0.644857,-0.288565,4.483302


In [22]:
selected_var = reg_2022.columns[2:]
selected_var

Index(['solo_30s_ratio', '네일숍', '미용실', '애완동물', '분식전문점', '한식음식점', '양식음식점',
       '일식음식점', '중식음식점', '치킨전문점', '커피-음료', '패스트푸드점', '제과점', '호프-간이주점', '대형마트',
       '쇼핑업체수', '편의점', '슈퍼마켓', '사업체수', '종사자수', '지가변동률', '지가', '대학교', '기타대학'],
      dtype='object')

In [23]:
### 분석 - OLS

selected_var = reg_2022.columns[len(depend_list):]

year_list = ['2022', '2023', '2024']

df_ols = pd.DataFrame()

for year in year_list:
    df = globals()[f'reg_{year}']
    for idx, depend in enumerate(depend_list):
        X = sm.add_constant(df[selected_var])

        # Fit the OLS model
        model = sm.OLS(df[depend], X)
        result = model.fit()

        summary = result.summary()

        temp = summary.tables[1].as_csv()

        ols_result = pd.read_csv(StringIO(temp), skipinitialspace=True)
        ols_result.columns = ['var', 'coef', 'std', 't', 'p', 'confi25', 'conf97']
        ols_result = ols_result[['var', 'coef', 'p']]
        ols_result['p'] = ols_result['p'].apply(lambda x: '+' if x < 0.05 else ' ')
        ols_result['coef'] = ols_result['coef'].apply(lambda x: round(x, 3))
        ols_result.columns = [f'var', f'c_{year}_{depend_name[idx]}', f'p_{year}_{depend_name[idx]}']
        ols_result['var'] = ols_result['var'].str.strip()

        globals()[f'ols_{year}_{depend}'] = ols_result

        if (year == '2022') & (idx == 0):
            df_ols = ols_result
        else:
            df_ols = df_ols.merge(ols_result,
                                        how = 'left',
                                        on = 'var')

In [24]:
ols_2022_solo_30s_ratio

,var,c_2022_30,p_2022_30
0,const,-0.000,
1,네일숍,0.453,+
2,미용실,-0.092,
3,애완동물,-0.104,
4,분식전문점,-0.308,
5,한식음식점,-0.262,
6,양식음식점,0.073,
7,일식음식점,0.685,+
8,중식음식점,-0.003,
9,치킨전문점,-0.317,+


In [25]:
df_ols.to_csv(f'{final_result}/ols_all.csv', index = False)
df_ols.to_excel(f'{final_result}/ols_all.xlsx', index = False)

In [26]:
len(reg_col)

27

## 컬럼명을 영어로 바꾸는 준비
- 전진선택법, 후진제거법은 변수명 영어로

In [27]:
reg_2022_en = reg_2022.copy()

new_columns = ['var{}'.format(i) for i in range(1,len(reg_col) - len(depend_list) )]
reg_2022_en.columns = depend_list + new_columns

In [28]:
col_dict = {}

for i in range(len(df_std.columns)):
    col_dict[reg_2022_en.columns[i]] = reg_2022.columns[i]
    col_dict[reg_2022.columns[i]] = reg_2022_en.columns[i]

col_dict.update({'Intercept' : '상수',
                 '상수' : 'intercept',
                 'const' : '상수',
                 '상수' : 'const'})

# 전진선택법

In [29]:
### 전진선택법 함수

def forward_selection(data, response):
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = float('inf'), float('inf')
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {}".format(response, ' + '.join(selected + [candidate]))
            score = sm.OLS.from_formula(formula, data).fit().aic
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates[0]
        if current_score > best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
        else:
            break
    formula = "{} ~ {}".format(response, ' + '.join(selected))
    model = sm.OLS.from_formula(formula, data).fit()
    return model


In [30]:
### 분석 - fwd

selected_var = reg_2022.columns[len(depend_list):]

df_fwd = pd.DataFrame()

for year in year_list:
    df = globals()[f'reg_{year}'].copy()
    df.columns = depend_list + new_columns

    for idx, depend in enumerate(depend_list):
        drop_list = [item for item in depend_list if item != depend]

        model = forward_selection(df.drop(columns = drop_list), depend)
        summary = model.summary()
        temp = summary.tables[1].as_csv()

        fwd_result = pd.read_csv(StringIO(temp), skipinitialspace=True)
        fwd_result.columns = ['var', 'coef', 'std', 't', 'p', 'confi25', 'conf97']

        fwd_result['var'] = fwd_result['var'].str.strip()
        fwd_result['coef'] = fwd_result['coef'].apply(lambda x: round(x, 3))
        fwd_result['var_mapped'] = fwd_result['var'].apply(lambda x: col_dict[x])
        fwd_result = fwd_result[['var', 'var_mapped', 'coef', 'p' ]]
        fwd_result['var_mapped'] = fwd_result['var_mapped'].str.strip()
        fwd_result['p'] = fwd_result['p'].apply(lambda x: '+' if x < 0.05 else ' ')
        fwd_result.columns = [f'var2', f'var',f'c_{year}_{depend_name[idx]}', f'p_{year}_{depend_name[idx]}']
        globals()[f'fwd_{year}_{depend}'] = fwd_result

        if (year == '2022') & (idx == 0):
            df_fwd = fwd_result
        else:
            df_fwd = df_fwd.merge(fwd_result,
                                        how = 'left',
                                        on = ['var', 'var2'])

In [31]:
df_fwd.to_csv(f'{final_result}/fwd_all.csv', index = False)
df_fwd.to_excel(f'{final_result}/fwd_all.xlsx', index = False)

In [32]:
df_fwd

,var2,var,c_2022_all,p_2022_all,c_2022_20,p_2022_20,c_2022_30,p_2022_30,c_2023_all,p_2023_all,c_2023_20,p_2023_20,c_2023_30,p_2023_30,c_2024_all,p_2024_all,c_2024_20,p_2024_20,c_2024_30,p_2024_30
0,Intercept,상수,-0.000,,0.000,,-0.000,,-0.000,,0.000,,0.000,,0.000,,-0.000,,-0.000,
1,var18,사업체수,0.251,+,NaN,NaN,NaN,NaN,0.266,+,NaN,NaN,NaN,NaN,0.553,+,0.228,+,NaN,NaN
2,var22,대학교,0.174,+,0.437,+,-0.065,,0.172,+,0.430,+,-0.076,,0.166,+,0.435,+,NaN,NaN
3,var6,양식음식점,0.314,+,0.123,,NaN,NaN,0.279,+,0.133,,0.151,,0.261,+,0.096,,0.160,+
4,var12,제과점,-0.378,+,-0.286,+,-0.564,+,-0.498,+,-0.237,+,-0.576,+,-0.326,+,NaN,NaN,-0.460,+
5,var16,편의점,0.338,+,0.371,+,0.429,+,0.533,+,0.522,+,0.561,+,0.466,+,0.389,+,0.324,+
6,var3,애완동물,-0.515,+,-0.468,+,NaN,NaN,-0.707,+,-0.630,+,NaN,NaN,-0.671,+,-0.701,+,NaN,NaN
7,var2,미용실,0.414,+,-0.085,,NaN,NaN,0.428,+,0.071,,-0.331,+,0.255,,NaN,NaN,-0.269,
8,var21,지가,-0.079,,NaN,NaN,-0.342,+,NaN,NaN,-0.007,,-0.040,,NaN,NaN,NaN,NaN,-0.164,+


### 후진제거법

In [33]:
def backward_elimination(data, response, significance_level=0.05):
    features = data.columns.tolist()
    features.remove(response)

    while len(features) > 0:
        features_with_constant = sm.add_constant(data[features])  # Add a constant term for the intercept
        model = sm.OLS(data[response], features_with_constant).fit()
        p_values = model.pvalues.iloc[1:]  # Get p-values for the features only
        max_p_value = p_values.max()  # Find the feature with the maximum p-value
        if max_p_value > significance_level:
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
        else:
            break

    model = sm.OLS(data[response], sm.add_constant(data[features])).fit()
    return model


In [34]:
### 분석 - bwd
selected_var = reg_2022.columns[len(depend_list):]

df_bwd = pd.DataFrame()

for year in year_list:
    df = globals()[f'reg_{year}'].copy()

    df.columns = depend_list + new_columns


    for idx, depend in enumerate(depend_list):
        drop_list = [item for item in depend_list if item != depend]

        model = backward_elimination(df.drop(columns = drop_list), depend)
        summary = model.summary()
        temp = summary.tables[1].as_csv()

        bwd_result = pd.read_csv(StringIO(temp), skipinitialspace=True)
        bwd_result.columns = ['var', 'coef', 'std', 't', 'p', 'confi25', 'conf97']

        bwd_result['var'] = bwd_result['var'].str.strip()
        bwd_result['coef'] = bwd_result['coef'].apply(lambda x: round(x, 3))
        bwd_result['var_mapped'] = bwd_result['var'].apply(lambda x: col_dict[x])
        bwd_result = bwd_result[['var', 'var_mapped', 'coef', 'p' ]]
        bwd_result['var_mapped'] = bwd_result['var_mapped'].str.strip()
        bwd_result['p'] = bwd_result['p'].apply(lambda x: '+' if x < 0.05 else ' ')
        bwd_result.columns = [f'var2', f'var',f'c_{year}_{depend_name[idx]}', f'p_{year}_{depend_name[idx]}']

        globals()[f'bwd_{year}_{depend}'] = bwd_result

        if (year == '2022') & (idx == 0):
            df_bwd = bwd_result
        else:
            df_bwd = df_bwd.merge(bwd_result,
                                        how = 'left',
                                       on = ['var', 'var2'])

In [35]:
df_bwd.to_csv(f'{final_result}/bwd_all.csv', index = False)
df_bwd.to_excel(f'{final_result}/bwd_all.xlsx', index = False)

In [36]:
df_bwd

,var2,var,c_2022_all,p_2022_all,c_2022_20,p_2022_20,c_2022_30,p_2022_30,c_2023_all,p_2023_all,c_2023_20,p_2023_20,c_2023_30,p_2023_30,c_2024_all,p_2024_all,c_2024_20,p_2024_20,c_2024_30,p_2024_30
0,const,상수,0.000,,0.000,,-0.000,,-0.000,,0.000,,0.000,,0.000,,-0.000,,-0.000,
1,var2,미용실,0.355,+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,var3,애완동물,-0.481,+,-0.531,+,NaN,NaN,-0.537,+,-0.501,+,NaN,NaN,-0.562,+,-0.676,+,NaN,NaN
3,var6,양식음식점,0.340,+,0.156,+,NaN,NaN,0.273,+,0.131,+,0.195,+,0.260,+,NaN,NaN,0.154,+
4,var12,제과점,-0.369,+,NaN,NaN,-0.573,+,-0.502,+,-0.357,+,-0.617,+,-0.365,+,NaN,NaN,-0.475,+
5,var16,편의점,0.353,+,0.338,+,0.381,+,0.573,+,0.358,+,0.498,+,0.570,+,0.376,+,0.261,+
6,var18,사업체수,0.232,+,NaN,NaN,NaN,NaN,0.227,+,NaN,NaN,NaN,NaN,0.462,+,0.246,+,NaN,NaN
7,var22,대학교,0.162,+,0.449,+,NaN,NaN,0.157,+,0.450,+,NaN,NaN,0.167,+,0.449,+,NaN,NaN


# 각연도별 통합

In [37]:
selected_var = reg_2022.columns[2:]

year_list = ['2022', '2023', '2024']

df_bwd = pd.DataFrame()

for year in year_list:

    for idx, depend in enumerate(depend_list):

        ols = globals()[f'ols_{year}_{depend}']
        fwd = globals()[f'fwd_{year}_{depend}']
        bwd = globals()[f'bwd_{year}_{depend}']

        ols['var'] = ols['var'].apply(lambda x: x if x != 'const' else '상수')

        temp = ols.merge(fwd.drop(columns = 'var2'),
                    how = 'outer',
                    on = 'var',
                    suffixes=('_ols', '_fwd'))
        temp = temp.merge(bwd.drop(columns = 'var2'),
                    how = 'outer',
                    on = 'var',
                    suffixes=('_fwd', '_bwd'))

        temp = temp.sort_values(by=[f'c_{year}_{depend_name[idx]}_fwd', f'p_{year}_{depend_name[idx]}'])
        temp = temp[temp['var'] != '상수']

        globals()[f'result_{year}_{depend}']= temp

        globals()[f'result_{year}_{depend}'].to_csv(f'{final_result}/{year}_{depend}.csv', index = False)
        globals()[f'result_{year}_{depend}'].to_excel(f'{final_result}/{year}_{depend}.xlsx', index = False)

        globals()[f'result_{year}_{idx}'] = temp

In [39]:
result_2022_1


,var,c_2022_20_ols,p_2022_20_ols,c_2022_20_fwd,p_2022_20_fwd,c_2022_20,p_2022_20
3,애완동물,-0.538,+,-0.468,+,-0.531,+
19,종사자수,-0.372,+,-0.329,+,-0.264,+
12,제과점,-0.222,,-0.286,+,NaN,NaN
2,미용실,-0.011,,-0.085,,NaN,NaN
14,대형마트,0.078,,0.067,,NaN,NaN
23,기타대학,0.085,,0.080,,NaN,NaN
20,지가변동률,0.109,,0.111,+,0.111,+
6,양식음식점,0.079,,0.123,,0.156,+
1,네일숍,0.298,+,0.266,+,0.186,+
7,일식음식점,0.384,+,0.278,,NaN,NaN
